In [1]:
import pandas
pandas.__version__

'0.23.4'

In [2]:
pandas.set_option('display.max_columns', None)
pandas.set_option('display.max_rows', None)

In [3]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
df = pandas.read_csv('../../v3_CSV/databases/connections_database.csv', 
                     header=None,
                     sep="\s*,\s*", engine='python',
                     names=['derivation name', 'linear index', 
                            'input type', 'in local ID', 'in global ID',
                            'output type', 'out local ID', 'out global ID'])

In [4]:
df['derivation name'] = df['derivation name'].str.replace('"', '')
df['input type'] = df['input type'].str.replace('"', '')
df['output type'] = df['output type'].str.replace('"', '')

In [5]:
df.shape

(490, 8)

In [6]:
df.head(23)

,derivation name,linear index,input type,in local ID,in global ID,output type,out local ID,out global ID
0,frequency relations,1,infrule,2303943,declareInitialExpr,expression,3293094,5900595848
1,frequency relations,2,infrule,304948,declareInitialExpr,expression,3294004,0404050504
2,frequency relations,3,infrule,8332941,declareInitialExpr,expression,921465,3131211131
3,frequency relations,4,infrule,9103516,declareInitialExpr,expression,8482459,3131111133
4,frequency relations,5,expression,8374556,3132131132,infrule,3832748,subRHSofExprXintoExprY
5,frequency relations,5,expression,3293094,5900595848,infrule,3832748,subRHSofExprXintoExprY
6,frequency relations,5,infrule,3832748,subRHSofExprXintoExprY,expression,8394853,0934990943
7,frequency relations,6,expression,3294004,0404050504,infrule,2939482,subRHSofExprXintoExprY
8,frequency relations,6,expression,8482459,3131111133,infrule,2939482,subRHSofExprXintoExprY
9,frequency relations,6,infrule,2939482,subRHSofExprXintoExprY,expression,3993940,1293923844


In [7]:
list_of_derivs = list(df['derivation name'].unique())

# single derivation; "expr local to global"

In [17]:
this_deriv_df = df[df['derivation name']==list_of_derivs[0]]

In [20]:
def get_expr_tuples_for_df(this_deriv_df):
    list_of_expr_tuples = []
    for indx, ths_row in this_deriv_df.iterrows():
        row_as_dict = ths_row.to_dict()
        if row_as_dict['input type'] == 'expression':
            expr_tuple = (str(row_as_dict['in local ID']), row_as_dict['in global ID'])
            list_of_expr_tuples.append(expr_tuple)
        if row_as_dict['output type'] == 'expression':
            expr_tuple = (str(row_as_dict['out local ID']), row_as_dict['out global ID'])
            list_of_expr_tuples.append(expr_tuple)
    return list(set(list_of_expr_tuples))

In [21]:
list_of_expr_tuples = get_expr_tuples_for_df(this_deriv_df)
list_of_expr_tuples

[('3293094', '5900595848'),
 ('8341200', '2131616531'),
 ('2934848', '3121513111'),
 ('9380032', '2113211456'),
 ('921465', '3131211131'),
 ('3294004', '0404050504'),
 ('8482459', '3131111133'),
 ('8394853', '0934990943'),
 ('8374556', '3132131132'),
 ('3993940', '1293923844')]

# single derivation; "derivations"

In [8]:
list_of_derivs[0]

'frequency relations'

In [9]:
this_deriv_df = df[df['derivation name']==list_of_derivs[0]]

In [10]:
def get_step_IDs(this_deriv_df):
    list_of_step_IDs = []
    for indx, ths_row in this_deriv_df.iterrows():
        row_as_dict = ths_row.to_dict()
        if row_as_dict['input type']=='infrule':
            list_of_step_IDs.append(row_as_dict['in local ID'])
        elif row_as_dict['output type']=='infrule':
            list_of_step_IDs.append(row_as_dict['out local ID'])

    return list(set(list_of_step_IDs))

In [11]:
list_of_step_IDs = get_step_IDs(this_deriv_df)

In [12]:
list_of_step_IDs

[9483715,
 9483843,
 2303943,
 8946983,
 3832748,
 8332941,
 304948,
 6822583,
 8345721,
 2939482,
 9103516]

In [13]:
def extract_printable_steps(list_of_step_IDs, this_deriv_df):
    dict_of_steps = {}
    for step_ID in list_of_step_IDs:
        step_str=""
        #print('step_ID =', step_ID)
        list_of_dicts_for_this_step = []
        for indx, ths_row in this_deriv_df.iterrows():
            #print(ths_row.to_dict())
            row_as_dict = ths_row.to_dict()
            if row_as_dict['input type'] == 'infrule' and row_as_dict['in local ID']==step_ID:
                list_of_dicts_for_this_step.append(row_as_dict)
            if row_as_dict['output type'] == 'infrule' and row_as_dict['out local ID']==step_ID:
                list_of_dicts_for_this_step.append(row_as_dict)

    #    print(list_of_dicts_for_this_step)

        for sub_dict in list_of_dicts_for_this_step:
            if sub_dict['input type']=='infrule':
                inf_rule = sub_dict['in global ID']
            if sub_dict['output type']=='infrule':
                inf_rule = sub_dict['out global ID']
        step_str+=("     '"+str(step_ID)+"': {'inf rule': '"+inf_rule+"',\n")
        
        list_of_input_expr_local_ID = []
        for sub_dict in list_of_dicts_for_this_step:
            if sub_dict['input type']=='expression':
                list_of_input_expr_local_ID.append(sub_dict['in local ID'])
        list_of_input_expr_local_ID = list(set(list_of_input_expr_local_ID))
        # https://stackoverflow.com/questions/3590165/join-a-list-of-items-with-different-types-as-string-in-python
        list_of_input_expr_local_ID = ', '.join("'"+str(x)+"'" for x in list_of_input_expr_local_ID)
        step_str+=("                 'inputs':  ["+str(list_of_input_expr_local_ID)+"],\n" )
        
        list_of_input_expr_local_ID = []
        for sub_dict in list_of_dicts_for_this_step:
            if sub_dict['input type']=='feed':
                list_of_input_expr_local_ID.append(sub_dict['in local ID'])
        list_of_input_expr_local_ID = list(set(list_of_input_expr_local_ID))
        list_of_input_expr_local_ID = ', '.join("'"+str(x)+"'" for x in list_of_input_expr_local_ID)
        step_str+=("                 'feeds':   ["+str(list_of_input_expr_local_ID)+"],\n" )

        list_of_input_expr_local_ID = []
        for sub_dict in list_of_dicts_for_this_step:
            if sub_dict['output type']=='expression':
                list_of_input_expr_local_ID.append(sub_dict['out local ID'])
        list_of_input_expr_local_ID = list(set(list_of_input_expr_local_ID))
        list_of_input_expr_local_ID = ', '.join("'"+str(x)+"'" for x in list_of_input_expr_local_ID)
        step_str+=("                 'outputs': ["+list_of_input_expr_local_ID+"],\n" )

        step_str+=("                 'linear index': "+str(list_of_dicts_for_this_step[0]['linear index'])+"},")

        dict_of_steps[list_of_dicts_for_this_step[0]['linear index']] = step_str

    return dict_of_steps

In [14]:
dict_of_steps = extract_printable_steps(list_of_step_IDs, this_deriv_df)

In [15]:
list_of_linindx = list(dict_of_steps.keys())
list_of_linindx.sort()
for linindx in list_of_linindx:
    print(dict_of_steps[linindx])

     '2303943': {'inf rule': 'declareInitialExpr',
                 'inputs':  [],
                 'feeds':   [],
                 'outputs': ['3293094'],
                 'linear index': 1},
     '304948': {'inf rule': 'declareInitialExpr',
                 'inputs':  [],
                 'feeds':   [],
                 'outputs': ['3294004'],
                 'linear index': 2},
     '8332941': {'inf rule': 'declareInitialExpr',
                 'inputs':  [],
                 'feeds':   [],
                 'outputs': ['921465'],
                 'linear index': 3},
     '9103516': {'inf rule': 'declareInitialExpr',
                 'inputs':  [],
                 'feeds':   [],
                 'outputs': ['8482459'],
                 'linear index': 4},
     '3832748': {'inf rule': 'subRHSofExprXintoExprY',
                 'inputs':  ['8374556', '3293094'],
                 'feeds':   [],
                 'outputs': ['8394853'],
                 'linear index': 5},
     '2939482

# all the derivations

In [24]:
for this_d in list_of_derivs:
    this_deriv_df = df[df['derivation name']==this_d]
    list_of_expr_tuples = get_expr_tuples_for_df(this_deriv_df)
    print('# '+this_d)
    for this_tup in list_of_expr_tuples:
        print("        '"+this_tup[0]+"': '"+this_tup[1]+"',")

# frequency relations
        '3293094': '5900595848',
        '8341200': '2131616531',
        '2934848': '3121513111',
        '9380032': '2113211456',
        '921465': '3131211131',
        '3294004': '0404050504',
        '8482459': '3131111133',
        '8394853': '0934990943',
        '8374556': '3132131132',
        '3993940': '1293923844',
# quadratic equation derivation
        '8811221': '9999999968',
        '8882098': '9285928292',
        '6608123': '5982958249',
        '909127': '8582954722',
        '2985412': '9385938295',
        '123901': '5928285821',
        '8761200': '9999999969',
        '173400': '5959282914',
        '112': '5928292841',
        '1212129': '5938459282',
        '6608102': '9582958294',
        '2657355': '5982958248',
        '4433112': '9582958293',
        '7777621': '5958392859',
# euler equation proof
        '1111289': '9848292229',
        '9984877': '9482943948',
        '3747585': '4928239482',
        '1029383': '9492920340',
       

In [16]:
for this_d in list_of_derivs:
    this_deriv_df = df[df['derivation name']==this_d]
    list_of_step_IDs = get_step_IDs(this_deriv_df)
    dict_of_steps = extract_printable_steps(list_of_step_IDs, this_deriv_df)
    list_of_linindx = list(dict_of_steps.keys())
    list_of_linindx.sort()
    print("  '"+this_d+"': {")
    for linindx in list_of_linindx:
        print(dict_of_steps[linindx])
    print("      },")

  'frequency relations': {
     '2303943': {'inf rule': 'declareInitialExpr',
                 'inputs':  [],
                 'feeds':   [],
                 'outputs': ['3293094'],
                 'linear index': 1},
     '304948': {'inf rule': 'declareInitialExpr',
                 'inputs':  [],
                 'feeds':   [],
                 'outputs': ['3294004'],
                 'linear index': 2},
     '8332941': {'inf rule': 'declareInitialExpr',
                 'inputs':  [],
                 'feeds':   [],
                 'outputs': ['921465'],
                 'linear index': 3},
     '9103516': {'inf rule': 'declareInitialExpr',
                 'inputs':  [],
                 'feeds':   [],
                 'outputs': ['8482459'],
                 'linear index': 4},
     '3832748': {'inf rule': 'subRHSofExprXintoExprY',
                 'inputs':  ['8374556', '3293094'],
                 'feeds':   [],
                 'outputs': ['8394853'],
                 'linea